In [1]:
import numpy as np 
import pandas as pd 
import yfinance as yf 
import datetime as dt 
from _measurements import * 
from _weight_optimization import *
from _time_varying_optimization import *
from _regime_detection import *
import seaborn as sns 
import pytse_client as tse 
import matplotlib.pyplot as plt
import os
sns.set_theme()

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# List of symbols
symbols = [
    'AAPL', 'MSFT', 'GOOGL', 
    'AMZN', "TSLA", 'NVDA', 
    'JNJ', 'WMT', 'KO', 
    'BAC', 'ADBE', 'MCD',
    'NFLX', 'NKE', 'PUM.DE',
    'PM', 'IBM', 'GE', "GM"
]

# Download historical data for the symbols
data = yf.download(symbols, end='2023-08-07')["Close"]

# Print the first few rows of the data
data.dropna(inplace=True)
returns = data.pct_change().dropna()
returns

[*********************100%%**********************]  19 of 19 completed


,AAPL,ADBE,AMZN,BAC,GE,GM,GOOGL,IBM,JNJ,KO,MCD,MSFT,NFLX,NKE,NVDA,PM,PUM.DE,TSLA,WMT
Date,,,,,,,,,,,,,,,,,,,
2010-11-19,-0.005512,0.006276,0.003959,-0.003419,0.011222,0.002047,-0.009605,0.004780,0.000000,0.007519,0.007846,-0.005805,0.027981,0.040500,0.032282,0.006056,0.004571,0.036801,0.007595
2010-11-22,0.021615,0.011088,0.033794,-0.030875,-0.011714,-0.005254,0.000660,0.002344,-0.003290,-0.000777,-0.001507,0.001557,0.088303,0.001282,0.000727,-0.005518,-0.011375,0.077767,-0.000184
2010-11-23,-0.014775,-0.033927,-0.012853,-0.018584,-0.016843,-0.024355,-0.013886,-0.015200,-0.011789,-0.010114,-0.006413,-0.023708,-0.003239,-0.007682,-0.023256,-0.011937,-0.023849,0.035030,-0.013056
2010-11-24,0.019661,0.007449,0.053805,0.017133,0.011421,0.006917,0.020514,0.018369,0.006680,0.015561,0.005949,0.009952,0.005647,0.014896,0.024554,0.007487,0.002572,0.026034,0.006335
2010-11-26,0.000635,0.011268,-0.000282,-0.014184,-0.008783,0.009558,-0.008353,-0.013099,-0.015642,-0.007739,-0.011827,-0.004730,0.016581,-0.006587,-0.012346,-0.007262,-0.002138,-0.004229,-0.004999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-31,0.003166,0.032711,0.011119,0.003135,-0.001311,0.008410,0.001056,0.005089,-0.039833,-0.008803,-0.002823,-0.007241,0.030978,0.016295,-0.000449,-0.001402,-0.000976,0.003716,-0.000313
2023-08-01,-0.004276,0.005365,-0.014886,-0.011875,-0.008929,-0.006516,-0.008816,-0.005895,0.008118,-0.002584,-0.007265,0.001250,-0.000797,-0.008968,-0.004751,-0.014741,-0.013021,-0.023782,-0.004692
2023-08-02,-0.015490,-0.034238,-0.026426,-0.013599,-0.011040,-0.019150,-0.024097,0.005861,0.006039,0.003076,-0.004982,-0.026283,-0.020336,-0.017276,-0.048122,-0.003155,-0.001979,-0.026660,0.000691


In [4]:
methods_mu = ["historical", "EWMA", "ledoit", "oracle", "shrunk", "mincovdet"]
methods_cov = ["historical", "EWMA", "ledoit", "oracle", "shrunk", "mincovdet"]
objectives = ["min_risk", "max_sharpe", "max_return", "max_utility"]

final_results = {}

for i in range(len(methods_mu)):
    mu = methods_mu[i]
    cov = methods_cov[i]
    for objective in objectives:
        params = {
            "train_low": 100, 
            "train_high": 1000, 
            "test_low": 100, 
            "test_high": 1000, 
            "step": 30,
            "method_mu": mu, 
            "method_cov": cov, 
            "objective": objective
            }
        
        int_opt = RollingTimeOptimization(returns, params)
        results = int_opt()
        print(f"{objective} - {mu} - {cov}")
        final_results[f"{objective} - {mu} - {cov}"] = results
        

min_risk - historical - historical


max_sharpe - historical - historical


Train Search Space: train interval = 820:  81%|████████  | 25/31 [05:29<01:22, 13.71s/it]

KeyboardInterrupt: 

In [ ]:
# data = (returns + 1).cumprod()
# rg = RegimeDetection(data)
# betas = rg.trend_filtering(returns*100, lambda_value=12)
# tr = rg.regime_switch_series(betas)